## Produce driver standings 

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions

###Find race years for which the data is to be processed

In [0]:
race_results_df = spark.read.format('delta').load(f'{presentation_folder_path}/race_results') \
    .filter(f"file_date = '{v_file_date}'") 

In [0]:
race_year_list = df_column_to_list(race_results_df, 'race_year')

In [0]:
from pyspark.sql.functions import col
race_results_df = spark.read.format('delta').load(f'{presentation_folder_path}/race_results') \
    .filter(col('race_year').isin(race_year_list))

In [0]:
#Si queremos contar el numero de victorias que tuvo cada piloto en la temporada, en este caso 2020, podemos usar la funcion when. 
#Cuando la condición se cumpla, devuelve 1. 

from pyspark.sql.functions import sum, when, col, count

driver_standings_df = race_results_df \
    .groupBy('race_year', 'driver_name', 'driver_nationality') \
    .agg(sum('points').alias('total_points'),
         count(when(col('position') == 1, True)).alias('wins'))
    

In [0]:
from pyspark.sql.window import Window 
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = driver_standings_df.withColumn('rank', rank().over(driver_rank_spec))


In [0]:
merge_condition = 'tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year'
merge_delta(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, merge_condition, 'race_year')